# <span style="color:#336699">SER-347 - Introdução à Programação para Sensoriamento Remoto</span>
<hr style="border:2px solid #0077b9;">

# <span style="color:#336699">Aula 22 - Leitura e Escrita de Dados Vetoriais</span>


[<img src="img/feature-collection.png" alt="Coleção de Feições" style="height: 75px;" align="right">](https://www.dpi.inpe.br)


- Gilberto Ribeiro de Queiroz
- Thales Sehn Körting
- Fabiano Morelli

# 1. Introdução
<hr style="border:1px solid #0077b9;">

Nesta parte da aula iremos discutir como realizar a leitura e escrita de dados geoespaciais representados na forma de uma **coleção de feições** (*feature collection*). A `Figura 1` mostra uma exemplo de coleção de feições, as unidades federativas do Brasil. 

<center><img src="img/feature-collection.png" alt="Coleção de Feições" width="480"><br>
**Figura 1** - Coleção de Feições</center>

Repare que essa forma de representação, em geral, assume que todos os objetos da coleção possuem uma estrutura comum, a qual denominamos de **esquema das feições**. O esquema das feições é formado pelo conjunto de atributos usados para caracterizar as feições do conjunto. Além disso, cada atributo é associado a um tipo de dado.

No caso da coleção mostrada na `Figura 1` o esquema denominado `unidades_federativas` possui o seguinte conjunto de atributos:
* `ufid`: código do IBGE, um número inteiro usado para identificar as unidades federativas.
* `nome`: nome da unidade federativa, uma string.
* `populacao`: total da população da unidade federativa, um número inteiro.
* `e_vida`: expectativa de vida, um número real.
* `fronteira`: limite da unidade federativa, representada por um tipo geométrico (`MultiPolygon`).

Atualmente existem inúmeros formatos para codificação de dados geoespaciais vetoriais: GeoJSON, GML, KML, ESRI Shapefile, Geomedia, Atlas BNA, entre outros. Além desses formatos de arquivo ainda encontramos dados geoespaciais armazenados e gerenciados por Sistemas Gerenciadores de Bancos de Dados (SGBD) como MySQL, PostgreSQL, IBM DB2, Oracle e Microsoft SQL Server. Outra forte tendência é a disponibilização de dados através de serviços Web, como o OGC WFS (Web Feature Service). Por conta disso, é importante usarmos tecnologias que nos abstraiam ao máximo dos detalhes de cada um desses formatos e sistemas.

Nesse sentido a biblioteca [GDAL](http://www.gdal.org) é um dos pilares básicos de praticamente todos os sistemas geoespaciais de software livre da atualidade. Em Python podemos utilizar essa biblioteca diretamente através de um *binding* (ou *wrapper*) que pode ser instalado com o gerenciador de pacotes da Anaconda ou pelo `pip`. 

Apesar da GDAL ser uma biblioteca robusta e completa, sua API Python expõe os tipos e operações com um estilo de programação muito próximo da linguagem C/C++, que é usada para seu desenvolvimento. Por isso, existem outras bibliotecas em Python que tentam fornecer um estilo de programação mais próximo dos idiomas usados em Python. Nesse sentido, a [Fiona](http://toblerity.org/fiona) é uma biblioteca que procura facilitar a leitura e escrita de dados geoespaciais adotando um estilo de programação mais próxima do universo Python.

Ao contrário da GDAL/OGR, que fornece um modelo de classes próprio (*DataSource*, *Layer*, *Feature*, *FeatureDefn*, *Geometry*) para acesso aos dados, a Fiona oferece esse suporte nos mesmos moldes dos protocolos de Entrada e Saída (E/S) da API de arquivos de Python (Python IO), além do uso de dicionários (*mappings*) e iteradores para acesso aos elementos do conjunto de dados. Por isso, nesta parte da aula iremos utilizar primeiramente a biblioteca [Fiona](http://toblerity.org/fiona) para realizar a leitura e escrita de arquivos ESRI Shapefile. Depois iremos ver como realizar processamentos equivalentes usando o *binding* Python da biblioteca GDAL.

Vale ressaltar que a Fiona é construída sobre a [OGR](http://www.gdal.org/ogr_apitut.html), parte da biblioteca [GDAL](http://www.gdal.org) voltada para tratamento de dados vetoriais. No entanto, ela não expõe nenhum dos tipos da GDAL. Os registros são mantidos em dicionários que seguem o formato [GeoJSON](http://geojson.org). Os tipos geométricos retornados pela Fiona podem ser facilmente convertidos em geometrias da biblioteca [Shapely](https://github.com/Toblerity/Shapely). Também é possível usar a [pyproj](https://github.com/jswhit/pyproj) e a [Rtree](http://toblerity.org/rtree) em conjunto com a Fiona.

# 2. Leitura e Escrita de Dados Vetoriais com a Fiona
<hr style="border:1px solid #0077b9;">

## 2.1. Instalando a Biblioteca Fiona
<hr style="border:0.25px solid #0077b9;">

Para instalar a Fiona, ative seu ambiente de trabalho da Anaconda e instale o pacote através do `conda`:
```bash
conda activate geospatial

conda install fiona
```

No meu ambiente, foi instalada a versão `1.7.10`.

## 2.2. Leitura de Dados
<hr style="border:0.25px solid #0077b9;">

Nesta seção iremos construir um pequeno programa que irá abrir um arquivo ESRI Shapefile contendo regiões indicadas pelo sistema DETER como possíveis áreas de desmatamento, como mostrado na `Figura 2`.

<table>
    <caption>**Figura 2** - Área de desmantamento apontadas pelo DETER (INPE).</caption>
    <tr>
        <td><img src="img/deter-mapa.png" alt="Regiões" width="380"></td>
        <td><img src="img/deter-atributos.png" alt="Regiões" width="300"></td>
    </tr>
</table>

Esse programa irá apresentar o esquema dessa coleção de feições e, em seguida, irá recuperar cada uma das feições, obtendo a data em que a região foi observada. Para cada região será computado o seu centróide.

O arquivo que usaremos se chama `Deter_201707.shp`:

In [ ]:
!ls -l dados

Para acessar as funcionalidades da biblioteca Fiona faça a seguinte importação:

In [ ]:
import fiona

fiona.__version__

Além disso, como iremos acessar as geometrias associadas às feições e extrair seu centróide, iremos precisar da biblioteca Shapely:

In [ ]:
from shapely.geometry import shape

A abertura do arquivo é realizada através da operação `fiona.open`.

Usaremos uma construção em Python que ainda não vimos no curso, com o uso do `with`, mas que basicamente associa o nome `deter` ao conteúdo do arquivo `Deter_201707.shp`, introduzindo um novo escopo:

In [ ]:
with fiona.open("dados/Deter_201707.shp", "r") as deter:
    
    # Número de feições
    num_features = len( deter )
    
    print( "Número de feições: {}\n". format( num_features ) )
    
    
    # Sistema de Referência Espacial
    crs = deter.crs
    
    print( "CRS: {}\n".format(crs) )
    
    
    # Extensão da coleção de feições
    mbr = deter.bounds
    
    print( "xmin: {}, xmax: {}".format(mbr[0], mbr[2]) )
    print( "ymin: {}, ymax: {}\n".format(mbr[1], mbr[3]) )
    
    
    # Esquema das feições
    for k, v in deter.schema["properties"].items():
        print( "Atributo: {}, Tipo: {}".format(k, v) )
        
    print("\nTipo do atributo geométrico: {}\n". format(deter.schema["geometry"]) )
    
    
    # Acessando cada uma das feições
    for feature in deter:
        # obtendo a geometria associada a feição
        geom = shape( feature["geometry"] )
        
        # obtendo o atributo de data associado a feição
        view_date = feature["properties"]["view_date"]
        
        # computando o centróide da geometria recuperada
        centroide = geom.centroid
        
        print( "view_date: {}, Localização: {}".format(view_date, centroide.wkt) )

## 2.3. Escrita de Dados
<hr style="border:0.25px solid #0077b9;">

Nesta seção iremos utilizar o mapa das unidades federativas brasileiras disponibilizado pelo IBGE, mostrado na `Figura 3`.

<table>
    <caption>**Figura 3** - Unidades Federativas (IBGE).</caption>
    <tr>
        <td><img src="img/uf-mapa.png" alt="Regiões" width="380"></td>
        <td><img src="img/uf-atributos.png" alt="Regiões" width="300"></td>
    </tr>
</table>

**Obs.:** Continuaremos esta seção na próxima aula! Veja a seção seguinte por hora!

# 3. Leitura e Escrita de Dados Vetoriais com a GDAL/OGR
<hr style="border:1px solid #0077b9;">

A parte da biblioteca GDAL voltada para manipulação de dados vetoriais é conhecida por OGR. A `Figura 5` apresenta as principais classes que compõem esta API.

<center>
    <img src="img/modelo-ogr.png" alt="Regiões" width="600"><br>
    **Figura 5** - Modelo de Objetos da OGR em Python.
</center>

A **classe `DataSource`** representa uma coleção de camadas de informação ou *layers*. Um objeto `DataSource` pode ser usado para acessar um único arquivo, um conjunto de arquivos, um conjunto de tabelas de um banco de dados, ou coleções de feições em um serviço Web do tipo OGC Web Feature Service (WFS).

A **classe `Layer`** representa uma camada de informação contida em um `DataSource`. Essa classe contém operações que possibilitam a leitura e escrita de feições (*features*), isto é, um objeto geográfico. Neste documento também usaremos o termo *coleção de feições* ou *feature collection* quando nos referirmos a uma camada de informação.

Todas as feições de uma mesma camada possuem a mesma estrutura, isto é, o mesmo conjunto de atributos. Denominamos esta estrutura da camada de informação de **esquema**. A **classe `FeatureDefn`** é usada para descrever o esquema de uma camada de informação, isto é, ela fornece uma lista com o nome dos atributos e seus respectivos tipos de dados. As feições pertencentes a uma mesma camada de informação irão compartilhar esta estrutura.

A **classe `Feature`** representa os dados de uma feição, que podem ser formadas por valores alfanuméricos e geométricos.

Para entender melhor esta estrutura (`Figura 5`), vamos considerar o caso de um mapa contendo informações dos municípios brasileiros no formato ESRI Shapefile. Neste caso, os dados estarão organizados em pelo menos três arquivos: um com a extensão `.dbf`, contendo os atributos alfanuméricos de cada feição; um segundo com a extensão `.shp`, contendo os polígonos representando as fronteiras dos municípios; e, um terceiro com a extensão `.shx`, cotendo os índices posicionais para rápido acesso às geometrias. Suponha que os arquivos possuam o nome base `municipios_brasileiros`. Um objeto do tipo `DataSource` (fonte de dados) irá representar esse conjunto de arquivos. A partir dele, teremos acesso ao conjunto de feições (municípios) através de um objeto do tipo `Layer` (camada de informação). Esse objeto por sua vez irá possibilitar acessar os dados de cada município, através de objetos do tipo `Feature` (feição geográfica). A partir de um objeto do tipo `Feature` podemos obter tanto os atributos alfanuméricos, como código do município, nome do município, e população total, quanto a geometria usada para representar seus limites (polígonos).

Ainda completam o diagrama da `Figura 5`, a **classe `FieldDefn`**, que representa as informações de um dado atributo do esquema da camada de informação; e, a **classe `Geometry`**, que representa os tipos geométricos suportados pela OGR, como pontos, linhas e polígonos.


**Observação:** O conjunto de arquivos Shapefile também pode incluir arquivos com extensões como `.prj`, contendo a descrição do sistema de referência espacial, e, `.cpg`, contendo a codificação dos caracteres dos dados contidos no arquivo `.dbf`. Para maiores detalhes do formato Esri Shapefile, veja a descrição fornecida na [Wikipedia](https://en.wikipedia.org/wiki/Shapefile).

## 3.1. Carregando a Biblioteca GDAL/OGR
<hr style="border:0.25px solid #0077b9;">

Uma boa prática ao trabalhar com bibliotecas que não fazem parte da distribuição padrão do Python consiste na verificação de sua carga logo após as instruções `import`. O trecho de código abaixo mostra como realizar essa verificação logo no início do seu script, para certificar que a biblioteca GDAL tenha sido devidamente carregada antes de prosseguir com as próximas instruções do programa. Neste script, utilizamos um bloco `try-except` para verificar se o módulo `osgeo` e as APIs `ogr` e `gdal` foram devidamente carregados (`linhas 3-6`). Na `linha 9` utilizamos a função `VersionInfo` da API GDAL para obter uma string com a versão e data de build da GDAL.

In [ ]:
import sys

try:
    from osgeo import gdal, ogr
except:
    sys.exit("ERRO: módulo GDAL/OGR não encontrado!")

# versão da biblioteca GDAL/OGR
print(gdal.VersionInfo("VERSION"))

## 3.2. Leitura de Dados
<hr style="border:0.25px solid #0077b9;">

Vamos abrir o Shapefile `Deter_201707.shp`:

In [ ]:
shp = ogr.Open("dados/Deter_201707.shp")

Veja que a operação `ogr.Open` cria um objeto chamado `shp` da classe `DataSource`:

In [ ]:
type(shp)

A partir de um objeto do tipo `DataSource` podemos recuperar a camada de informação contida nessa fonte atravé do método `GetLayer`:

In [ ]:
layer = shp.GetLayer("Deter_201707")

type(layer)

De um objeto do tipo `Layer` podemos recuperar algumas infromações básicas, como mostrado abaixo:

In [ ]:
nome_layer = layer.GetName()
print("Layer: ", nome_layer)

bbox = layer.GetExtent()
print("\tExtensão.......: ", bbox)

crs = layer.GetSpatialRef().ExportToWkt()
print("\tSRS............: ", crs)

tipo_geometrias = layer.GetGeomType()
print("\tTipo Geométrico: ", tipo_geometrias)

print("\tPolígonos? ", tipo_geometrias == ogr.wkbPolygon)

num_features = layer.GetFeatureCount()
print("\t#Feições.......: ", num_features)

Podemos também obter o esquema das feições da camada:

In [ ]:
layer_def = layer.GetLayerDefn()

print("Name       - Type    Width  Precision")

for i in range(layer_def.GetFieldCount()):
    field_name =  layer_def.GetFieldDefn(i).GetName()
    field_type_code = layer_def.GetFieldDefn(i).GetType()
    field_type = layer_def.GetFieldDefn(i).GetFieldTypeName(field_type_code)
    field_width = layer_def.GetFieldDefn(i).GetWidth()
    field_precision = layer_def.GetFieldDefn(i).GetPrecision()

    print(field_name.ljust(10) + " - " + \
          field_type.ljust(7) + " " + \
          str(field_width).ljust(6) + " " + \
          str(field_precision))

Para acessar os elementos da camada podemos utilizar um laço do tipo `for` como o mostrado abaixo:

In [ ]:
for feature in layer:
    view_date = feature.GetField("view_date")

    geom = feature.GetGeometryRef()
    
    centroide = geom.Centroid()
    
    print( "view_date: {}, Localização: {}".format(view_date, centroide.ExportToWkt()) )

# Considerações Finais
<hr style="border:1px solid #0077b9;">

Quando você escrever scripts da GDAL/OGR que manipulem diversos arquivos e camadas antes de terminar, faça a limpeza explícita dos objetos através de comandos como `del ds` ou `del layer` para garantir que os recursos usados sejam devolvidos. O binding da GDAL/OGR não permite usarmos a estratégia do comando composto `with` pois ele não impelementa inicialização/finalização de recursos. 

A Fiona pode ser usada com o comando composto `with`.

Uma ferramenta útil para análise de dados em Python é o GeoPandas, que será tópico de discussão da nossa última aula.

# Referências Bibliográficas
<hr style="border:1px solid #0077b9;">

* [The Fiona User Manual](http://toblerity.org/fiona/manual.html). Acesso em: 24 de Maio de 2018.<br><br>

* [GDAL/OGR Cookbook](https://pcjericks.github.io/py-gdalogr-cookbook/). Acesso em: 24 de Maio de 2018.